---
# 신경망 종합 테스트
- burst 하여 active cell 이 많으면 새로운 패턴을 인지한다는 의미
- 먼저 spatial pooler 가 안정되어야 한다.
---
encoder - sp - tm -> prediction + textual representation<br/>
예측 cell / 활성화 cell 비교 - 예측이 맞는지 확인 가능<br/>
예측 column / 입력 column 비교 - 마찬가지<br/>
burst 한 column / 활성화 된 column - 마찬가지<br/>


In [1]:
import numpy as np
import random
import nbimporter
import Encoder
import SP3
import TM

Importing Jupyter notebook from Encoder.ipynb
Importing Jupyter notebook from SP3.ipynb
Importing Jupyter notebook from TM.ipynb
Importing Jupyter notebook from memoryStruct.ipynb


In [2]:
encoder = Encoder.ScalarEncoder(out_size=100, w=20, max_val=30)
sp = SP3.SpatialPooler(encoder.out_size, columns=512, perm_conn=.6, minOver=encoder.w/5, potential_rate=0.9)

In [3]:
for epoch in range(10):
    a = np.arange(encoder.max_val)
    random.shuffle(a)
    
    for i in a:
        input_data = encoder.encode(i)
        sp.compute_SDR(input_data)

In [10]:
for i in range(encoder.max_val):
    sp.compute_SDR(encoder.encode(i), learn=False)
    sdr1 = sp.getActiveColumns()
    sp.compute_SDR(encoder.encode(i+1), learn=False)
    sdr2 = sp.getActiveColumns()
    
    sp.similarity(sdr1, sdr2, False)
    print('')

유사도 : 0.8
[  7  28  34  48 230 234 336 408 421 486]
[ 28  34  48  72 213 234 336 408 421 486]

유사도 : 0.3
[ 28  34  48  72 213 234 336 408 421 486]
[  6   7  34  55  82 139 213 320 400 486]

유사도 : 0.4
[  6   7  34  55  82 139 213 320 400 486]
[  6  55  82 213 286 319 343 448 476 494]

유사도 : 0.6
[  6  55  82 213 286 319 343 448 476 494]
[  6  15  82 186 194 319 343 442 448 476 503]

유사도 : 0.36363636363636365
[  6  15  82 186 194 319 343 442 448 476 503]
[125 143 146 186 323 394 422 448 460 476 503]

유사도 : 0.36363636363636365
[125 143 146 186 323 394 422 448 460 476 503]
[ 21  31 125 146 281 311 381 394 395 460]

유사도 : 0.6
[ 21  31 125 146 281 311 381 394 395 460]
[ 31  55 125 146 200 281 381 390 424 460]

유사도 : 0.3
[ 31  55 125 146 200 281 381 390 424 460]
[ 55 159 163 191 223 254 326 348 383 390 399 424 432 455 492]

유사도 : 0.4666666666666667
[ 55 159 163 191 223 254 326 348 383 390 399 424 432 455 492]
[  5  12  75 104 154 159 189 191 254 259 323 328 348 420 424 432 433 492]

유사도 : 0.44

In [241]:
tm = TM.TemporalMemory(sdr_size=sp.columnCount, threshold=2)

In [244]:
tm.learn = True

seq_data = [1,2,3,4,]

for i in range(10):
    for s in seq_data:
        dat = encoder.encode(s)
        sp.compute_SDR(dat, learn=False)
        sdr = sp.getActiveColumns_raw()
        tm.compute(sdr)

In [245]:
tm.learn = False

test_data = [1,2,3,4]
sub = [4,3]

for d in test_data:
    dat = encoder.encode(d)
    sp.compute_SDR(dat, learn=False)
    sdr = sp.getActiveColumns_raw()
    tm.compute(sdr)

    print("현재 sdr : {}".format(sp.getActiveColumns()))
    print("다음 예측 : {}".format(tm.column_predicted()))
    print("현재 burst : {}".format(tm.column_bursted()))
    print("\n")

현재 sdr : [ 46  54 112 207 251 345 354 357 363 434]
다음 예측 : [354, 47, 112, 144, 18, 115, 443, 54, 311, 251, 510]
현재 burst : [54, 354, 46]


현재 sdr : [ 18  47  54 112 115 251 311 354 443 510]
다음 예측 : [13, 334, 176, 241, 18, 112, 144, 115, 54, 311, 151, 443]
현재 burst : []


현재 sdr : [ 13  79 112 115 144 151 176 241 311 334]
다음 예측 : [104, 301, 13, 79, 144, 241, 115, 278, 440, 446]
현재 burst : [79]


현재 sdr : [ 13  79 104 115 144 241 278 301 440 446]
다음 예측 : [354, 357, 363, 301, 46, 207, 112, 241, 434, 13, 79, 54, 345, 251, 446]
현재 burst : []




In [15]:
# len(tm.connInfo.totalSegments)
cell = 2000
print('{} 번 cell 에 연결되어 있는 synapse : {}'.format(cell, tm.connInfo.totalCells[cell].connectedSynapses))
#print(len(tm.prev_activeCells))
#print(len(tm.activeCells))
#for c in tm.connInfo.totalCells:
 #   if(len(c.connectedSynapses) > 0):
  #      print("살아있는 cell : " + str(c.cell_id))
        
#choose = list(tm.connInfo.totalCells[cell].connectedSynapses)[0]
#for i in tm.connInfo.totalSegments:
 #   for j in i.synapses:
  #      if (j.synapse_id == choose):
   #         print(j.synapse_id)
    #        print(j.preSynapticCell)
     #       print(j.permanence)
        
print(sp.getActiveColumns())
print(tm.predicted)
print(tm.bursted)

2000 번 cell 에 연결되어 있는 synapse : set()
[ 86 160 176 237 353 361 415 458 492 500]
[131, 285, 329, 361, 176, 415, 86, 361, 458, 492, 237, 353, 285, 462, 40, 285, 361, 40, 145, 451, 462, 500, 40]
[86, 237, 353, 361, 458, 492]
